In [1]:
import matplotlib.pyplot as mp
import pandas as pd
import seaborn as sb
from sklearn.model_selection import train_test_split  # Import train_test_split here
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

# Import file with data
data = pd.read_csv("small1.csv")

selectedFeatures = ['Shoe up angle', 'Down tube front diameter', 'Shoe sideways', 'Front  fender width', 'Fit scheme']
extractedData = data[selectedFeatures]
# print(extractedData.head())
# print(extractedData.describe())

trainData, validationData = train_test_split(extractedData, test_size=0.2, random_state=42)
trainFeatures = trainData[['Shoe up angle', 'Down tube front diameter', 'Shoe sideways', 'Front  fender width']]
validationFeatures = validationData[['Shoe up angle', 'Down tube front diameter', 'Shoe sideways', 'Front  fender width']]

# Encode the labels (1-hot representation)
label_encoder = LabelEncoder()
trainTarget = label_encoder.fit_transform(trainData['Fit scheme'])
trainLabels = tf.keras.utils.to_categorical(trainTarget, len(label_encoder.classes_))
validationTarget = label_encoder.transform(validationData['Fit scheme'])
validationlabels = tf.keras.utils.to_categorical(validationTarget, len(label_encoder.classes_))


In [2]:
# design an ANN
modelANN = tf.keras.Sequential([
  tf.keras.layers.Dense(32, input_dim = 4, activation = tf.nn.relu),
  tf.keras.layers.Dense(4, activation=tf.nn.softmax),
])

modelANN.compile(optimizer=tf.keras.optimizers.SGD(),
               loss=tf.keras.losses.CategoricalCrossentropy(),
               metrics=['accuracy'])
modelANN.summary()

C:\Users\laura\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 292 (1.14 KB)

 Trainable params: 292 (1.14 KB)

 Non-trainable params: 0 (0.00 B)

In [3]:
# train the ANN
print('# Fit model on training data ')
history = modelANN.fit(trainFeatures, trainLabels, batch_size=10, epochs=3)
# print('\nhistory dict:', history.history)

# Fit model on training data 
Epoch 1/3
8/8 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - accuracy: 0.4143 - loss: 32.1495
Epoch 2/3
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7316 - loss: 0.8745
Epoch 3/3
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.7820 - loss: 0.6197


In [4]:
predictedProbabilities = modelANN.predict(validationFeatures)
print(predictedProbabilities)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 862ms/step
[[0.00201845 0.140746   0.8268593  0.03037634]
 [0.00201845 0.140746   0.8268593  0.03037634]
 [0.00491085 0.55704355 0.36700323 0.07104231]
 [0.00201845 0.140746   0.8268593  0.03037634]
 [0.00201845 0.140746   0.8268593  0.03037634]
 [0.00512339 0.55166304 0.3709825  0.07223105]
 [0.00201845 0.140746   0.8268593  0.03037634]
 [0.00267612 0.21365114 0.74341005 0.04026272]
 [0.00139311 0.07065304 0.9081562  0.01979761]
 [0.00267612 0.21365114 0.74341005 0.04026272]
 [0.00139311 0.07065304 0.9081562  0.01979761]
 [0.00201845 0.140746   0.8268593  0.03037634]
 [0.00201845 0.140746   0.8268593  0.03037634]
 [0.00201845 0.140746   0.8268593  0.03037634]
 [0.00201845 0.140746   0.8268593  0.03037634]
 [0.00201845 0.140746   0.8268593  0.03037634]
 [0.00139311 0.07065304 0.9081562  0.01979761]
 [0.00267612 0.21365114 0.74341005 0.04026272]
 [0.00491084 0.5570436  0.3670033  0.07104229]
 [0.00491084 0.5570436  0.3670033  0.07104229]]


In [7]:
import numpy as np

predictedLabels = np.argmax(predictedProbabilities, axis=1)
print(predictedLabels)

[2 2 1 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 1 1]


In [ ]:
# validate the trained ANN
performanceMetrics = modelANN.evaluate(validationFeatures, validationlabels)
print(performanceMetrics)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 492ms/step - accuracy: 0.7000 - loss: 0.7528
